In [1]:
import numpy as np
import os
import sys
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from itertools import combinations
import ase
from ase import io
from copy import deepcopy

In [6]:
# counter = 0
separation = 3.5
FG_numatoms_dict = { # p1 is the vertex from which p2 and p3 are drawn
    'T': {
        'nat': 7,
        'p1': -1,
        'p2': 2,
        'p3': 3
        },
    'TT': {
        'nat': 10,
        'p1': -1,
        'p2': -2,
        'p3': 2
        },
    'DPP': {
        'nat': 18,
        'p1': 6,
        'p2': 5,
        'p3': 1
        },
}
operating_path = r'/Users/connorganley/Library/Mobile Documents/com~apple~CloudDocs/Documents/JHU/2023-2024/polymer-builder/code/input-data/all-polymer-combos/unopt-xyz/'
# Thiophene (T) rings have 6 atoms each 
for xyzfile in os.listdir(operating_path):
    name = xyzfile[:-4].split('-')
    # print(name)
    # add code here to tabulate the polymer xyz file's composition by FG index; e.g. DPP is indices 1-18 (0-17), T is indices 19-25 (18-24), etc.; will be expandable for EDOT etc.
    # if name[0] in ['T', 'TT']:
    #     if name[2] in ['T', 'TT']:
    #         if name[3] in ['T', 'TT']: #, 'EDOT', 'MEET']:
    #             if name[4] in ['T', 'TT']: #, 'EDOT', 'MEET']:
    for solvent_species in ['tpfb']:#'1,2,4-trimethylbenzene', '2-hexanone', '2-propanol', 'acetone', 'acetonitrile', 'acetophenone', 'anisole', 'benzaldehyde', 'carbon disulfide', 'chlorobenzene', 'chloroform', 'cyclohexane', 'cyclohexanone', 'dmf', 'dmso', 'ethanol', 'ethylbenzene', 'hexane', 'mesitylene', 'methanol', 'o-dichlorobenzene', 'o-xylene', 'tetrahydrofuran', 'thiophene', 'toluene', 'water', 'f4tcnq', 'f6tcnnq']:
        
        # Reading in polymer and solvent xyz files
        atom_index_counter = 0
        polymer = ase.io.read(operating_path + xyzfile)
        polymer_solvent = deepcopy(polymer)
        print(f'Polymer Name: {xyzfile[:-4]}')
        solvent = ase.io.read(operating_path + '../../all-solvents/unopt-xyz/' + solvent_species + '.xyz')
        solvent_copy = deepcopy(solvent)
        solvent_center = np.mean(solvent.get_positions(), axis=0)
        solvent_atoms_dist_to_center = solvent.get_positions() - solvent_center
        print(f'Solvent Name: {solvent_species}')

        # ---

        # DPP
        FG_DPP = polymer.get_positions()[atom_index_counter:atom_index_counter+FG_numatoms_dict[name[1]]['nat']]
        atom_index_counter += FG_numatoms_dict[name[1]]['nat']
        DPP_v13 = polymer.get_positions()[FG_numatoms_dict[name[1]]['p3']] - polymer.get_positions()[FG_numatoms_dict[name[1]]['p1']]
        DPP_v12 = polymer.get_positions()[FG_numatoms_dict[name[1]]['p2']] - polymer.get_positions()[FG_numatoms_dict[name[1]]['p1']]
        DPP_normalvec = np.cross(DPP_v13, DPP_v12) # create a normal vector to the FG plane
        DPP_normalvec = np.abs(DPP_normalvec / np.linalg.norm(DPP_normalvec)) # scale normal vector into unit vector
        DPP_center = np.mean([polymer.get_positions()[FG_numatoms_dict[name[1]]['p1']], polymer.get_positions()[FG_numatoms_dict[name[1]]['p2']], polymer.get_positions()[FG_numatoms_dict[name[1]]['p3']]], axis=0)
        
        solvent_position = DPP_center + separation*DPP_normalvec
        placement = solvent_atoms_dist_to_center + solvent_position
        solvent.set_positions(placement)

        for entry in solvent:
            polymer_solvent.append(entry)
        polymer_solvent.write(f'polymer-adspecies-interactions/unopt-xyz/{xyzfile[:-4]}-{solvent_species}.xyz')
        polymer_solvent = deepcopy(polymer)
        solvent = solvent_copy

        print(f'Wrote file {xyzfile[:-4]}-{solvent_species}.xyz')

                        # ---

                        # # Functional Group A1
                        # FG_A1 = polymer.get_positions()[atom_index_counter:atom_index_counter+FG_numatoms_dict[name[0]]['nat']]
                        # atom_index_counter += FG_numatoms_dict[name[0]]['nat']
                        # A1_v13 = FG_A1[FG_numatoms_dict[name[0]]['p3']] - FG_A1[FG_numatoms_dict[name[0]]['p1']]
                        # A1_v12 = FG_A1[FG_numatoms_dict[name[0]]['p2']] - FG_A1[FG_numatoms_dict[name[0]]['p1']]
                        # A1_normalvec = np.cross(A1_v13, A1_v12)
                        # A1_normalvec = np.abs(A1_normalvec / np.linalg.norm(A1_normalvec))
                        # A1_center = np.mean([FG_A1[FG_numatoms_dict[name[0]]['p1']], FG_A1[FG_numatoms_dict[name[0]]['p2']], FG_A1[FG_numatoms_dict[name[0]]['p3']]], axis=0)

                        # solvent_position = A1_center + separation*A1_normalvec
                        # placement = solvent_atoms_dist_to_center + solvent_position
                        # solvent.set_positions(placement)

                        # for entry in solvent:
                        #     polymer_solvent.append(entry)
                        # polymer_solvent.write(f'{xyzfile[:-4]}-{solvent_species}-A1.xyz')
                        # polymer_solvent = deepcopy(polymer)
                        # solvent = solvent_copy

                        # print(f'Wrote file {xyzfile[:-4]}-{solvent_species}-A1.xyz')
                        
                        # # ---

                        # # Functional Group A2
                        # FG_A2 = polymer.get_positions()[atom_index_counter:atom_index_counter+FG_numatoms_dict[name[2]]['nat']]
                        # atom_index_counter += FG_numatoms_dict[name[2]]['nat']
                        # A2_v13 = FG_A2[FG_numatoms_dict[name[2]]['p3']] - FG_A2[FG_numatoms_dict[name[2]]['p1']]
                        # A2_v12 = FG_A2[FG_numatoms_dict[name[2]]['p2']] - FG_A2[FG_numatoms_dict[name[2]]['p1']]
                        # A2_normalvec = np.cross(A2_v13, A2_v12)
                        # A2_normalvec = np.abs(A2_normalvec / np.linalg.norm(A2_normalvec))
                        # A2_center = np.mean([FG_A2[FG_numatoms_dict[name[2]]['p1']], FG_A2[FG_numatoms_dict[name[2]]['p2']], FG_A2[FG_numatoms_dict[name[2]]['p3']]], axis=0)

                        # solvent_position = A2_center + separation*A2_normalvec
                        # placement = solvent_atoms_dist_to_center + solvent_position
                        # solvent.set_positions(placement)

                        # for entry in solvent:
                        #     polymer_solvent.append(entry)
                        # polymer_solvent.write(f'{xyzfile[:-4]}-{solvent_species}-A2.xyz')
                        # polymer_solvent = deepcopy(polymer)
                        # solvent = solvent_copy

                        # print(f'Wrote file {xyzfile[:-4]}-{solvent_species}-A2.xyz')
                        
                        # # ---

                        # # Functional Group B
                        # FG_B = polymer.get_positions()[atom_index_counter:atom_index_counter+FG_numatoms_dict[name[3]]['nat']]
                        # atom_index_counter += FG_numatoms_dict[name[3]]['nat']
                        # B_v13 = FG_B[FG_numatoms_dict[name[3]]['p3']] - FG_B[FG_numatoms_dict[name[3]]['p1']]
                        # B_v12 = FG_B[FG_numatoms_dict[name[3]]['p2']] - FG_B[FG_numatoms_dict[name[3]]['p1']]
                        # B_normalvec = np.cross(B_v13, B_v12)
                        # B_normalvec = np.abs(B_normalvec / np.linalg.norm(B_normalvec))
                        # B_center = np.mean([FG_B[FG_numatoms_dict[name[3]]['p1']], FG_B[FG_numatoms_dict[name[3]]['p2']], FG_B[FG_numatoms_dict[name[3]]['p3']]], axis=0)

                        # solvent_position = B_center + separation*B_normalvec
                        # placement = solvent_atoms_dist_to_center + solvent_position
                        # solvent.set_positions(placement)

                        # for entry in solvent:
                        #     polymer_solvent.append(entry)
                        # polymer_solvent.write(f'{xyzfile[:-4]}-{solvent_species}-B.xyz')
                        # polymer_solvent = deepcopy(polymer)
                        # solvent = solvent_copy

                        # print(f'Wrote file {xyzfile[:-4]}-{solvent_species}-B.xyz')
                        
                        # # ---

                        # # Functional Group C
                        # FG_C = polymer.get_positions()[atom_index_counter:atom_index_counter+FG_numatoms_dict[name[4]]['nat']]
                        # atom_index_counter += FG_numatoms_dict[name[4]]['nat']
                        # C_v13 = FG_C[FG_numatoms_dict[name[4]]['p3']] - FG_C[FG_numatoms_dict[name[4]]['p1']]
                        # C_v12 = FG_C[FG_numatoms_dict[name[4]]['p2']] - FG_C[FG_numatoms_dict[name[4]]['p1']]
                        # C_normalvec = np.cross(C_v13, C_v12)
                        # C_normalvec = np.abs(C_normalvec / np.linalg.norm(C_normalvec))
                        # C_center = np.mean([FG_C[FG_numatoms_dict[name[4]]['p1']], FG_C[FG_numatoms_dict[name[4]]['p2']], FG_C[FG_numatoms_dict[name[4]]['p3']]], axis=0)

                        # solvent_position = C_center + separation*C_normalvec
                        # placement = solvent_atoms_dist_to_center + solvent_position
                        # solvent.set_positions(placement)

                        # for entry in solvent:
                        #     polymer_solvent.append(entry)
                        # polymer_solvent.write(f'{xyzfile[:-4]}-{solvent_species}-C.xyz')
                        # polymer_solvent = deepcopy(polymer)
                        # solvent = solvent_copy

                        # print(f'Wrote file {xyzfile[:-4]}-{solvent_species}-C.xyz')

                    # break
                    # select solvent molecules: water, n-hexane, acetonitrile, mesitylene
                    # identify where to place them above polymer FGs


Polymer Name: T-DPP-T-NDT-TT
Solvent Name: tpfb
Wrote file T-DPP-T-NDT-TT-tpfb.xyz
Polymer Name: TT-DPP-TT-TT-T6C
Solvent Name: tpfb
Wrote file TT-DPP-TT-TT-T6C-tpfb.xyz
Polymer Name: T-DPP-T-EDOT-BZ
Solvent Name: tpfb
Wrote file T-DPP-T-EDOT-BZ-tpfb.xyz
Polymer Name: TT-DPP-TT-EDOT-BZ
Solvent Name: tpfb
Wrote file TT-DPP-TT-EDOT-BZ-tpfb.xyz
Polymer Name: BZ-DPP-BZ-EDOT-T6C
Solvent Name: tpfb
Wrote file BZ-DPP-BZ-EDOT-T6C-tpfb.xyz
Polymer Name: BZ-DPP-BZ-NDT-T6C
Solvent Name: tpfb
Wrote file BZ-DPP-BZ-NDT-T6C-tpfb.xyz
Polymer Name: EDOT-DPP-EDOT-EDOT-BZ
Solvent Name: tpfb
Wrote file EDOT-DPP-EDOT-EDOT-BZ-tpfb.xyz
Polymer Name: T-DPP-T-T6C-T
Solvent Name: tpfb
Wrote file T-DPP-T-T6C-T-tpfb.xyz
Polymer Name: BZ-DPP-BZ-MEET-MEET
Solvent Name: tpfb
Wrote file BZ-DPP-BZ-MEET-MEET-tpfb.xyz
Polymer Name: T-DPP-T-BZ-NDT
Solvent Name: tpfb
Wrote file T-DPP-T-BZ-NDT-tpfb.xyz
Polymer Name: EDOT-DPP-EDOT-T-T6C
Solvent Name: tpfb
Wrote file EDOT-DPP-EDOT-T-T6C-tpfb.xyz
Polymer Name: BZ-DPP-BZ-EDOT-

## Solvent Pairwise Distances

In [ ]:
pairwise_distances = pdist(scaled)
distance_matrix = squareform(pairwise_distances)

pairs = list(combinations(range(len(solvent_data)), 2))

pairwise_distances_with_indices = list(zip(pairs, pairwise_distances))

with open('solvent-pairwise-distances-labeled.txt', 'w') as f:
    for (i, j), distance in pairwise_distances_with_indices:
        f.write(f'{i} {j} {distance}\n')
f.close()
plt.hist(pairwise_distances)
plt.xlabel('Euclidean Pairwise Distance')
plt.ylabel('Frequency')
plt.title('Pairwise distances between Solvents (7 features)')
# print(distance_matrix)